In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Cursor
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

#Criar um DB para armazenar tabelas dos times
engine = create_engine('sqlite:////workspaces/Visualiza--o-Dados-Premier-League/0_Bases_Tratadas/premier.db', echo =True)

connection = engine.connect()

#Preparar os dados para serem armazenados em um DB

dados = pd.read_csv('/workspaces/Visualiza--o-Dados-Premier-League/1_Bases_Originais/premier_league_24_25.csv', sep = ';')

In [2]:
#Adicionar uma nova coluna com o vencedor/perdedor

def resultado (time):

    if time['Gols_Casa'] > time['Gols_Visita']:
        return time['Time_Casa']
    elif time['Gols_Casa'] < time['Gols_Visita']:
        return time['Time_Visita']
    else:
        return 'Empate'
    
dados['Vencedor'] = dados.apply(resultado, axis=1)

dados['Time_Casa'] = dados["Time_Casa"].str.replace(' ', '')
dados['Time_Visita'] = dados["Time_Visita"].str.replace(' ', '')
dados['Vencedor'] = dados['Vencedor'].str.replace(' ','')

dados.to_sql('Dados', con  = engine, if_exists='replace', index=False)


Teams = dados['Time_Casa'].unique()
dfteams = pd.DataFrame(Teams, columns= ['Times'])
dfteams = dfteams.sort_values(by=['Times'], ascending= True)
dfteams = dfteams.reset_index(drop=True)
dfteams


2025-12-15 12:47:49,282 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-15 12:47:49,289 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Dados")
2025-12-15 12:47:49,289 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,312 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Dados")
2025-12-15 12:47:49,313 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,314 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-12-15 12:47:49,314 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,316 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-12-15 12:47:49,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,317 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Dados")
2025-12-15 12:47:49,318 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:4

,Times
0,Arsenal
1,AstonVilla
2,Bournemouth
3,Brentford
4,Brighton
5,Chelsea
6,CrystalPalace
7,Everton
8,Fulham
9,Ipswich


In [3]:
#Criar tabelas unicas para cada time para que possam ser tratados de forma individual

for i in dfteams['Times'].unique():
    jogos_casa = dados[dados['Time_Casa'] == i]
    jogos_fora = dados[dados['Time_Visita'] == i]
    tabela_time = pd.concat([jogos_casa, jogos_fora])
    tabela_time.reset_index(drop=True, inplace=True)
    tabela_time.to_sql(i, con  = engine, if_exists='replace', index=False)

#Criar tabelas unicas por Rodada 1 - Atual

for j, inicio in enumerate(range(0, len(dados), 10), start=1):
    
    rodadas = dados.iloc[inicio:inicio+10]
    nome_rodada = f"Rodada {j}" 
    rodadas.to_sql(nome_rodada, con  = engine, if_exists='replace', index=False)


dfteams["Posição"] = range(1, 21)
dfteams["Times"] = dados['Time_Casa'].unique()
dfteams["Partidas_Jogadas"] = 0
dfteams["Partidas_Vencidas"] = 0
dfteams["Empates"] = 0
dfteams["Partidas_Perdidas"] = 0
dfteams["Gols_Marcados"] = 0
dfteams["Gols_Sofridos"] = 0
dfteams["Saldo_de_Gols"] = dfteams["Gols_Marcados"] - dfteams["Gols_Sofridos"]
dfteams["Impedimentos"] = 0
dfteams["Faltas_Cometidas"] = 0
dfteams["Faltas_Sofridas"] = 0
dfteams["Pontos"] = 0
dfteams = dfteams.sort_values(by= ["Pontos", "Saldo_de_Gols", "Gols_Marcados","Gols_Sofridos","Impedimentos","Faltas_Sofridas","Faltas_Cometidas"], ascending= [False,False,False,True,True,False,True])
dfteams = dfteams.reset_index(drop=True)
dfteams = dfteams.set_index("Posição")

2025-12-15 12:47:49,400 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-15 12:47:49,406 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Arsenal")
2025-12-15 12:47:49,407 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,408 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Arsenal")
2025-12-15 12:47:49,409 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,410 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-12-15 12:47:49,410 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,411 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-12-15 12:47:49,412 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 12:47:49,413 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Arsenal")
2025-12-15 12:47:49,414 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 1

In [4]:
display(dfteams)

,Times,Partidas_Jogadas,Partidas_Vencidas,Empates,Partidas_Perdidas,Gols_Marcados,Gols_Sofridos,Saldo_de_Gols,Impedimentos,Faltas_Cometidas,Faltas_Sofridas,Pontos
Posição,,,,,,,,,,,,
1,ManUtd,0,0,0,0,0,0,0,0,0,0,0
2,Ipswich,0,0,0,0,0,0,0,0,0,0,0
3,Arsenal,0,0,0,0,0,0,0,0,0,0,0
4,Everton,0,0,0,0,0,0,0,0,0,0,0
5,Newcastle,0,0,0,0,0,0,0,0,0,0,0
6,Nott'mForest,0,0,0,0,0,0,0,0,0,0,0
7,WestHam,0,0,0,0,0,0,0,0,0,0,0
8,Brentford,0,0,0,0,0,0,0,0,0,0,0
9,Chelsea,0,0,0,0,0,0,0,0,0,0,0


In [10]:
for x in range(1, 39):
    time = pd.read_sql(f"SELECT * FROM 'Rodada {x}'", con = engine)

    for _, row in time.iterrows():
        time_casa = row["Time_Casa"]
        time_visita = row["Time_Visita"]
        gols_casa = row["Gols_Casa"]
        gols_visita = row["Gols_Visita"]
        impedimentos_casa = row["impedimentos_casa"]
        impedimentos_visita = row["impedimentos_visita"]
        faltas_sofridas =row["faltas_cometidas_visita"]
        faltas_cometidas = row["faltas_sofridas_casa"]
        vencedor = row["Vencedor"]
        #loop do vencedor
        if vencedor == time_casa:  # Time da casa venceu
            dfteams.loc[dfteams["Times"] == time_casa, "Pontos"] += 3
        elif vencedor == time_visita:  # Time visitante venceu
            dfteams.loc[dfteams["Times"] == time_visita, "Pontos"] += 3
        elif vencedor == "Empate":  # Empate
            dfteams.loc[dfteams["Times"] == time_casa, "Pontos"] += 1
            dfteams.loc[dfteams["Times"] == time_visita, "Pontos"] += 1
        #loop dos gols marcados e sofridos
        if time_casa in dfteams["Times"].values:
            dfteams.loc[dfteams["Times"] == time_casa, "Gols_Marcados"] += gols_casa
            dfteams.loc[dfteams["Times"] == time_casa, "Gols_Sofridos"] += gols_visita
            dfteams.loc[dfteams["Times"] == time_visita, "Gols_Sofridos"] += gols_casa
            dfteams.loc[dfteams["Times"] == time_visita, "Gols_Marcados"] += gols_visita
            dfteams["Saldo_de_Gols"] = dfteams["Gols_Marcados"] - dfteams["Gols_Sofridos"]
        #loop de impedimentos
        if time_casa in dfteams["Times"].values:
            dfteams.loc[dfteams["Times"] == time_casa, "Impedimentos"] += impedimentos_casa
            dfteams.loc[dfteams["Times"] == time_visita, "Impedimentos"] += impedimentos_visita
        #loop de faltas
        if time_casa in dfteams["Times"].values:
            dfteams.loc[dfteams["Times"] == time_casa, "Faltas_Cometidas"] += faltas_cometidas
            dfteams.loc[dfteams["Times"] == time_visita, "Faltas_Sofridas"] += faltas_sofridas
        #loop de partidas jogadas
        if time_casa in dfteams["Times"].values:
            dfteams.loc[dfteams["Times"] == time_casa, "Partidas_Jogadas"] += 1
            dfteams.loc[dfteams["Times"] == time_visita, "Partidas_Jogadas"] += 1
        #loop de partidas perdidas, vencidas ou empates
        if time_casa in dfteams["Times"].values:
            dfteams.loc[dfteams["Times"] == vencedor, "Partidas_Vencidas"] += 1
        if row["Vencedor"] == "Empate":
            time_casa = row["Time_Casa"]
            time_visita = row["Time_Visita"]

            # Incrementar partidas empatadas para time_casa
            if time_casa in dfteams["Times"].values:
                dfteams.loc[dfteams["Times"] == time_casa, "Empates"] += 1

        # Incrementar partidas empatadas para time_visitante
            if time_visita in dfteams["Times"].values:
                dfteams.loc[dfteams["Times"] == time_visita, "Empates"] += 1
        if row["Vencedor"] == time_casa:
            dfteams.loc[dfteams["Times"] == time_visita, "Partidas_Perdidas"] += 1
        if row["Vencedor"] == time_visita:
            dfteams.loc[dfteams["Times"] == time_casa, "Partidas_Perdidas"] += 1
        
        
        
        rodada_class = f"Classificação {x}"
        dfteams = dfteams.sort_values(by= ["Pontos", "Saldo_de_Gols", "Gols_Marcados","Gols_Sofridos","Impedimentos","Faltas_Sofridas","Faltas_Cometidas"], ascending= [False,False,False,True,True,False,True])
        dfteams = dfteams.reset_index(drop=True)
       
        dfteams.to_sql(rodada_class, con  = engine, if_exists='replace', index=False)
    
    #rodada_atual = rodada_class.replace('Classificação', '').replace(' ','')
    #rodada_atual = int(rodada_atual)
    #dfteams = dfteams.sort_values(by= ["Pontos", "Saldo_de_Gols", "Gols_Marcados","Gols_Sofridos","Impedimentos","Faltas_Sofridas","Faltas_Cometidas"], ascending= [False,False,False,True,True,False,True])
    #dfteams = dfteams.reset_index(drop=True)
       
    #dfteams.to_sql(rodada_atual, con  = engine, if_exists='replace', index=False)

2025-12-15 13:11:40,712 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-15 13:11:40,713 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM 'Rodada 1'")
2025-12-15 13:11:40,715 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 13:11:40,715 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM 'Rodada 1'")
2025-12-15 13:11:40,716 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 13:11:40,717 INFO sqlalchemy.engine.Engine SELECT * FROM 'Rodada 1'
2025-12-15 13:11:40,719 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 13:11:40,722 INFO sqlalchemy.engine.Engine ROLLBACK
2025-12-15 13:11:40,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-15 13:11:40,747 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Classificação 1")
2025-12-15 13:11:40,748 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-15 13:11:40,749 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Classificação 1")
2025-12-15 13:11:40,749 INFO sqlalchemy.engine